In [166]:
"""Overall Summary of the Application
This application is a fraud detection and analysis system using Neo4j's Graph Data Science (GDS) library. It focuses on detecting fraudulent activities by analyzing relationships between users, devices, credit cards, and transactions in a graph database.

The application:

1) Connects to a Neo4j database (frauddb) to analyze financial transactions and user behavior.
2) Projects in-memory graphs using Neo4j's GDS library for running graph algorithms efficiently.
3) Computes centrality metrics (degree centrality) to identify users and entities (e.g., devices, IPs, and cards) that are highly connected and potentially high-risk.
4) Flags fraudulent users based on their transaction history and connections, assigning them the FlaggedUser label.
5) Calculates fraud ratios for various financial entities, such as cards and devices, determining the proportion of fraudulent activity within each category.
6) Applies the Louvain Community Detection algorithm to group users into communities based on their interactions, helping uncover fraud rings or coordinated attack groups.
7) Analyzes and ranks communities based on fraud prevalence, identifying clusters with a high proportion of fraudulent activity.
8) Outputs fraud-related insights, including flagged user ratios, community detection results, and risk scores for different financial identifiers.

This system is designed to identify fraudulent behavior, detect hidden fraud rings, and prioritize high-risk entities for investigation, helping prevent financial fraud and reduce risks in financial transactions.

"""

"Overall Summary of the Application\nThis application is a fraud detection and analysis system using Neo4j's Graph Data Science (GDS) library. It focuses on detecting fraudulent activities by analyzing relationships between users, devices, credit cards, and transactions in a graph database.\n\nThe application:\n\n1) Connects to a Neo4j database (frauddb) to analyze financial transactions and user behavior.\n2) Projects in-memory graphs using Neo4j's GDS library for running graph algorithms efficiently.\n3) Computes centrality metrics (degree centrality) to identify users and entities (e.g., devices, IPs, and cards) that are highly connected and potentially high-risk.\n4) Flags fraudulent users based on their transaction history and connections, assigning them the FlaggedUser label.\n5) Calculates fraud ratios for various financial entities, such as cards and devices, determining the proportion of fraudulent activity within each category.\n6) Applies the Louvain Community Detection algo

In [2]:
import pandas as pd
import configparser
import os
from neo4j import GraphDatabase, basic_auth
from graphdatascience import GraphDataScience
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 50)

c:\Users\mattd\anaconda3\envs\fraud-detection-demo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [146]:
print("hello world")

hello world


In [3]:
NEO4J_PROPERTIES_FILE = 'resources/aura-p2p-fd.ini'

In [4]:
## Using an ini file for credentials, otherwise providing defaults
HOST = 'neo4j://localhost'
USERNAME = 'neo4j'
PASSWORD = 'password'

if NEO4J_PROPERTIES_FILE is not None and os.path.exists(NEO4J_PROPERTIES_FILE):
    config = configparser.RawConfigParser()
    config.read(NEO4J_PROPERTIES_FILE)
    HOST = config['NEO4J']['HOST']
    USERNAME = config['NEO4J']['USERNAME']
    PASSWORD = config['NEO4J']['PASSWORD']
    DATABASE = config['NEO4J'].get('DATABASE', 'frauddb')

    print('Using custom database properties')
else:
    print('Could not find database properties file, using defaults')

Using custom database properties


In [5]:
# Use Neo4j URI and credentials according to your setup
# gds = GraphDatabase.driver(HOST, auth=basic_auth(USERNAME, PASSWORD))
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), database="frauddb")

In [10]:
def clear_graph_by_name(g_name):
    """
    Safely clears a projected graph in the Neo4j Graph Data Science (GDS) library.

    Parameters:
    g_name (str): The name of the projected graph to be cleared.

    Steps:
    1. Check if the graph with the given name exists in GDS.
    2. If the graph exists, retrieve it.
    3. Drop (delete) the graph from memory to free up resources.
    """
    
    # Step 1: Check if the graph exists in the GDS catalog
    if gds.graph.exists(g_name).exists:
        # Step 2: Retrieve the existing graph object
        g = gds.graph.get(g_name)
        # Step 3: Drop the graph to free up memory and prevent conflicts with new projections
        gds.graph.drop(g)

def clear_all_graphs():
    g_names = gds.graph.list().graphName.tolist()
    for g_name in g_names:
        g = gds.graph.get(g_name)
        gds.graph.drop(g)

def identifier_degrees(user_label, degree_property):
    """
    Projects a subgraph in Neo4j's Graph Data Science (GDS) library, computes degree centrality,
    and writes the computed property back to the database.

    Degree centrality is a graph algorithm that measures the number of direct connections
    (edges) a node has to other nodes.  It helps identify highly connected nodes in a network.
    
    Parameters:
    user_label (str): The label of the user nodes to include in the projection (e.g., 'User' or 'FlaggedUser').
    degree_property (str): The name of the property where the degree centrality values will be stored.

    Steps:
    1. Define a unique name for the projected graph (`g_name`).
    2. Clear any existing projected graph with the same name to prevent conflicts.
    3. Create a new graph projection, including specified node labels and relationships.
    4. Compute degree centrality for the projected graph and store it as `degree_property`.
    5. Write the computed centrality values back to the Neo4j database for specific node types.
    6. Drop the projected graph to free up memory.
    """

    # Step 1: Define a unique graph projection name
    # A graph projection in GDS library is a simplified in-memory representation of your database's 
    # graph that is optimized for graph algorithms 
    g_name = 'id-projection'

    # Step 2: Clear any existing graph with the same name to avoid conflicts
    clear_graph_by_name(g_name)

    # Step 3: Project a graph with specific node labels and relationship types
    # user_label is a dynamic variable representing a type of user node that will be 
    # included in the graph projection.
    # In Python, _ (underscore) is commonly used as a convention to indicate that a returned value is intentionally being ignored.
    g, _ = gds.graph.project(
        g_name, 
        [user_label, 'Card', 'Device', 'IP'],  # Nodes included in the projection
        {
            'HAS_CC': {'orientation': 'REVERSE'},  # Reverse the relationship direction for analysis
            'HAS_IP': {'orientation': 'REVERSE'},
            'USED': {'orientation': 'REVERSE'}
        }
    )

    # Step 4: Compute degree centrality and store it in `degree_property`
    # higher degree -> more connections -> more influence or risk
    # lower degree -> fewer connections -> less influence or risk
    # mutate computes the degree centrality for each node in the in-memory projection g
    # stores computed values in a new property (degree_property) inside the projection
    # modifies the in-memory graph but doesn't yet write to the database
    # degree_property = 'degree' or 'flagged_degree'
    gds.degree.mutate(g, mutateProperty=degree_property)

    # Step 5: Write the computed centrality values back to the Neo4j database
    # g is the in-memory projected graph where the degree centrality was computed
    # degree_property is the property to write (e.g. degree or flagged_degree).  
    # it stores the number of direct connections
    # ['Card', 'Device', 'IP'] is a list of node labels to which the property will be written
    # In graph theory, the degree of a node is the number of direct connetcions (edges) it has to other nodes
    gds.graph.writeNodeProperties(g, [degree_property], ['Card', 'Device', 'IP'])

    # Step 6: Drop the projected graph to free up memory
    g.drop()


In [6]:
import numpy
print(numpy.__version__)

1.26.4


In [152]:
# APOC (Awesome Procedures on Cypher).
# meta.stats() procedure, which provides metadata about the Neo4j database.
# YIELD labels extracts only the labels field from the apoc.meta.stats() result.
# UNWIND keys(labels) AS nodeLabel – Convert Dictionary to Rows
# Without UNWIND, the data remains a single row:
# {User: 100, Card: 200, Device: 300, IP: 400}
# UNWIND Allows Individual Access to Each Label

# total node counts
gds.run_cypher('''
    CALL apoc.meta.stats()
    YIELD labels
    UNWIND keys(labels) AS nodeLabel
    RETURN nodeLabel, labels[nodeLabel] AS nodeCount
''', database='frauddb')

,nodeLabel,nodeCount
0,User,33732
1,FlaggedUser,241
2,Device,51451
3,Card,118818
4,IP,585855


In [153]:
gds.run_cypher('''
    CALL apoc.meta.stats()
    YIELD relTypesCount
    UNWIND keys(relTypesCount) AS relationshipType
    RETURN relationshipType, relTypesCount[relationshipType] AS relationshipCount
''', database='frauddb')

,relationshipType,relationshipCount
0,USED,55026
1,HAS_IP,1488949
2,HAS_CC,128066
3,REFERRED,1870
4,P2P,102832


In [154]:
gds.run_cypher('''
    MATCH(u:User) RETURN u.fraudMoneyTransfer AS fraudMoneyTransfer, count(u) AS cnt
''', database='frauddb')

,fraudMoneyTransfer,cnt
0,0,33491
1,1,241


In [155]:
# This query identifies all User nodes involved in fraudulent money transfers, assigns them the FlaggedUser label, and returns the count of affected users.

gds.run_cypher('''
    MATCH(u:User) WHERE u.fraudMoneyTransfer=1 SET u:FlaggedUser RETURN count(u)
''', database='frauddb')

,count(u)
0,241


In [156]:
# Use GDS degree centrality to count the number of Users connected to each identifier type - Card, Device, IP
identifier_degrees('User', 'degree')
# Use GDS degree centrality to count the number of FLAGGED Users connected to each identifier type - Card, Device, IP
identifier_degrees('FlaggedUser', 'flaggedDegree')

# Calculate the ratio of flagged users to total users
gds.run_cypher('''
    MATCH(n) WHERE n:Card OR n:Device OR n:IP
    SET n.flaggedRatio = toFloat(n.flaggedDegree)/toFloat(n.degree)
''')

""


In [157]:
print('Flagged User Ratio for Card Count')
gds.run_cypher('''
MATCH (n:Card)
WHERE n.degree > 1
WITH toFloat(count(n)) AS total  // ✅ Calculate total count first

MATCH (n:Card)
WHERE n.degree > 1
WITH 
    CASE 
        WHEN n.flaggedRatio = 0 THEN '0'
        WHEN n.flaggedRatio = 1 THEN '1'
        ELSE 'Between 0-1' 
    END AS flaggedUserRatio, 
    total, 
    count(n) AS count  // ✅ Grouping on flaggedUserRatio

RETURN 
    flaggedUserRatio, 
    count, 
    round(toFloat(count)/total, 3) AS percentCount
ORDER BY flaggedUserRatio;

''')

Flagged User Ratio for Card Count


,flaggedUserRatio,count,percentCount
0,0,8185,0.964
1,1,31,0.004
2,Between 0-1,274,0.032


In [ ]:
"""
    Why is This Useful in Fraud Detection?
    - Identifies Devices Linked to Fraudsters - Helps find shared devices between flagged & unflagged users.
    - Detects High-Risk Devices - If many flagged users share the same device, it's suspicious.
    - Enhances Anomaly Detection - Helps monitor fraud rings using common devices.
"""

print('Flagged User Ratio for Device Count')
gds.run_cypher('''
    MATCH (n:Device) 
    WHERE n.degree > 1
    WITH toFloat(count(n)) AS total  // Compute total first

    MATCH (n:Device) 
    WHERE n.degree > 1
    WITH n, total, 
        CASE 
            WHEN n.flaggedRatio = 0 THEN '0'
            WHEN n.flaggedRatio = 1 THEN '1'
            ELSE 'Between 0-1' 
        END AS flaggedUserRatio
    WITH flaggedUserRatio, total, count(n) AS count  // Group results before return
    RETURN flaggedUserRatio, count, round(toFloat(count)/total, 3) AS percentCount
    ORDER BY flaggedUserRatio;
''')

Flagged User Ratio for Device Count


,flaggedUserRatio,count,percentCount
0,0,2655,0.967
1,1,2,0.001
2,Between 0-1,88,0.032


In [ ]:
# This projects a graph for community detection using the Louvain algorithm and 
# stores the detected communities in the Neo4j database.

clear_graph_by_name('comm-projection')

# Create a Graph Projection for Community Detection
"""
'comm-projection'	Name of the in-memory graph projection.
['User', 'Card', 'Device']	Includes only these node types in the graph projection.
{ 'HAS_CC': {...}, 'USED': {...}, 'P2P': {...} }	Includes only these relationships, with specific orientations.
"""

"""
Why Use UNDIRECTED for HAS_CC and USED?

Fraud analysis often needs to analyze shared resources (e.g., multiple users using the same card/device).
Making these relationships undirected allows algorithms to group entities together, regardless of who initiated the connection.
📌 Why Use NATURAL for P2P?

P2P (person-to-person transactions) often have natural directionality (sender → receiver).
Keeping this directionality allows Louvain community detection to identify fraud rings.
"""
g, _ = gds.graph.project('comm-projection', ['User','Card', 'Device'], {
    'HAS_CC': {'orientation': 'UNDIRECTED'},
    'USED': {'orientation': 'UNDIRECTED'},
    'P2P': {'orientation': 'NATURAL', 'aggregation': 'SINGLE'}
})

"""
Runs the Louvain algorithm on the in-memory graph g.
Finds communities (clusters) of closely connected nodes.
Stores the computed community ID in the database under the property louvainCommunityId.
"""
df = gds.louvain.write(g, writeProperty='louvainCommunityId')
g.drop()
df

writeMillis                                                                                                                                                                                                                                                                                                                                                                                                                 156
nodePropertiesWritten                                                                                                                                                                                                                                                                                                                                                                                                    204001
modularity                                                                                                                                                              

In [ ]:
print("Louvain Communities Ordered by count of Flagged Users")

"""
1) This retrieves all User nodes in the database.
Each User has already been assigned a Louvain community (louvainCommunityId) from the Louvain community detection algorithm.

2) Groups all users by louvainCommunityId (each user belongs to a detected fraud community).

3) Compute the Ratio of Fraudsters in Each Community

4) Sort and Limit to the Top 100 Fraudulent Communities

Why is This Useful for Fraud Detection?
- Identifies Fraud Rings - Communities with high fraud ratios are likely organized fraud groups.
- Prioritizes Investigations - Sorting by fraud count ensures investigators focus on the worst offenders first.
- Detects Hidden Patterns - Users may not be directly connected but belong to the same fraudulent network.

"""

gds.run_cypher('''
    MATCH (u:User)
    WITH u.louvainCommunityId AS community,
        count(u) AS cnt,
        sum(u.fraudMoneyTransfer) as flaggedCount
    RETURN community,
        cnt,
        flaggedCount,
        toFloat(flaggedCount)/toFloat(cnt) AS flaggedRatio
    ORDER BY flaggedCount DESC LIMIT 100
''')

Louvain Communities Ordered by count of Flagged Users


,community,cnt,flaggedCount,flaggedRatio
0,200458,155,7,0.045161
1,182733,7,4,0.571429
2,188313,6,3,0.500000
3,173038,240,3,0.012500
4,201222,7,3,0.428571
...,...,...,...,...
95,172511,3,1,0.333333
96,174011,20,1,0.050000
97,173401,5,1,0.200000
98,172844,2,1,0.500000


In [ ]:
PART II

In [21]:
# P2P with shared card rule

# This Cypher query identifies user pairs (u1, u2) who have engaged in 
# person-to-person (P2P) transactions and also share at least one credit card. 
# It then creates a new relationship (P2P_WITH_SHARED_CARD) between them to indicate 
# potential fraudulent collaboration.

""" 
gds.run_cypher('''
    -- Step 1: Identify User Pairs with P2P Transactions
    MATCH (u1:User)-[r:P2P]->(u2)
    WITH u1, u2, count(r) AS cnt  -- ✅ Count the number of P2P transactions between each pair

    -- Step 2: Check if These Users Share a Credit Card
    MATCH (u1)-[:HAS_CC]->(n)<-[:HAS_CC]-(u2)  -- ✅ Find shared credit cards between users
    WITH u1, u2, count(DISTINCT n) AS cnt  -- ✅ Count the number of shared credit cards

    -- Step 3: Create a New Relationship Indicating Shared Credit Card Usage
    MERGE (u1)-[s:P2P_WITH_SHARED_CARD]->(u2)  -- ✅ Create or merge the relationship

    -- Step 4: Return the Total Number of New Relationships Created
    RETURN count(DISTINCT s) AS cnt
''')
"""

gds.run_cypher('''
    MATCH (u1:User)-[r:P2P]->(u2)
    WITH u1, u2, count(r) AS cnt
    MATCH (u1)-[:HAS_CC]->(n)<-[:HAS_CC]-(u2)
    WITH u1, u2, count(DISTINCT n) AS cnt
    MERGE(u1)-[s:P2P_WITH_SHARED_CARD]->(u2)
    RETURN count(DISTINCT s) AS cnt
''')

,cnt
0,6240


In [23]:
# This Cypher query identifies user pairs (u1, u2) who share multiple financial and 
# device identifiers (credit cards, devices, or IP addresses) and creates a SHARED_IDS 
# relationship between them if they share more than two unique identifiers. This helps 
# detect potential duplicate or synthetic identities.

"""
gds.run_cypher('''
    -- Step 1: Find Users Sharing Credit Cards or Devices
    MATCH (u1:User)-[:HAS_CC|USED]->(n)<-[:HAS_CC|USED]-(u2)
    WHERE n.degree <= 10  -- ✅ Only consider entities (cards/devices) used by 10 or fewer users (limits noise)
    AND id(u1) < id(u2)  -- ✅ Ensures each pair is processed once (avoids duplicate relationships)

    -- Step 2: Count the Number of Shared Credit Cards/Devices
    WITH u1, u2, count(DISTINCT n) AS cnt

    -- Step 3: Find Additional Shared Identifiers (Credit Cards, Devices, IPs)
    MATCH (u1)-[:HAS_CC|USED|HAS_IP]->(m)<-[:HAS_CC|USED|HAS_IP]-(u2)
    WITH u1, u2, count(DISTINCT m) AS cnt  -- ✅ Count all shared identifiers

    -- Step 4: Create a Relationship if Users Share More than 2 Identifiers
    WHERE cnt > 2  -- ✅ Threshold: Only link users with more than 2 shared identifiers
    MERGE (u1)-[s:SHARED_IDS]->(u2)  -- ✅ Create or merge `SHARED_IDS` relationship

    -- Step 5: Return the Number of New Relationships Created
    RETURN count(DISTINCT s)
''')
"""

# Shared ids rule
gds.run_cypher('''
    MATCH (u1:User)-[:HAS_CC|USED]->(n)<-[:HAS_CC|USED]-(u2)
    WHERE n.degree <= 10 AND id(u1) < id(u2)
    WITH u1, u2, count(DISTINCT n) as cnt
    MATCH (u1)-[:HAS_CC|USED|HAS_IP]->(m)<-[:HAS_CC|USED|HAS_IP]-(u2)
    WITH u1, u2, count(DISTINCT m) as cnt
    WHERE cnt > 2
    MERGE(u1)-[s:SHARED_IDS]->(u2)
    RETURN count(DISTINCT s)
''')

,count(DISTINCT s)
0,5316


In [24]:
# This code creates a graph projection using only User nodes and runs Weakly 
# Connected Components (WCC) to identify groups of interconnected users based 
# on shared identifiers (SHARED_IDS) and financial links (P2P_WITH_SHARED_CARD). 
# The WCC results are written back to the database to assign a unique community ID 
# (wccId) to each connected group.

# Step 1: Clear Any Existing Graph Projection
clear_graph_by_name('comm-projection')  # ✅ Ensures we start fresh by removing old projections

# Step 2: Create an In-Memory Graph Projection for User Relationships
g, _ = gds.graph.project(
    'comm-projection',  # ✅ Name of the in-memory graph projection
    ['User'],  # ✅ Only includes User nodes (no Card, Device, or IP nodes)
    {
        'SHARED_IDS': {'orientation': 'UNDIRECTED'},  # ✅ Links users sharing financial/device identifiers
        'P2P_WITH_SHARED_CARD': {'orientation': 'UNDIRECTED'}  # ✅ Links users transacting with shared credit cards
    }
)

# Step 3: Run Weakly Connected Components (WCC) to Identify Connected Groups
df = gds.wcc.write(g, writeProperty='wccId')  # ✅ Assigns each connected user group a `wccId`

# Step 4: Drop the Graph Projection to Free Memory
g.drop()

# Step 5: Return WCC Results
df  # ✅ Outputs results (modularity, component count, computation time, etc.)


writeMillis                                                                                                                                                                                                                                                                                                                   76
nodePropertiesWritten                                                                                                                                                                                                                                                                                                      33732
componentCount                                                                                                                                                                                                                                                                                                             28203
componentDistribution                

In [25]:
# This Cypher query identifies all users who belong to communities (WCC groups) 
# that contain flagged users and marks them as FraudRiskUser by assigning a new 
# label and a fraudRisk=1 property. This helps flag potential fraudsters based on 
# their connections to known fraudulent accounts.

"""
gds.run_cypher('''
    -- Step 1: Collect WCC IDs of Flagged Users
    MATCH (f:FlaggedUser)  -- ✅ Retrieve all users who have been previously flagged as fraudulent
    WITH collect(DISTINCT f.wccId) AS flaggedCommunities  -- ✅ Get a unique list of their community IDs (wccId)

    -- Step 2: Find All Users in These Flagged Communities
    MATCH (u:User) 
    WHERE u.wccId IN flaggedCommunities  -- ✅ Identify users in the same WCC communities as flagged users

    -- Step 3: Mark These Users as High-Risk
    SET u:FraudRiskUser  -- ✅ Assign the "FraudRiskUser" label
    SET u.fraudRisk = 1  -- ✅ Explicitly mark them as high risk

    -- Step 4: Return the Total Number of Users Marked as Fraud Risk
    RETURN count(u)  -- ✅ Shows how many users were flagged as "FraudRiskUser"
''')

"""

gds.run_cypher('''
    MATCH (f:FlaggedUser)
    WITH collect(DISTINCT f.wccId) AS flaggedCommunities
    MATCH(u:User) WHERE u.wccId IN flaggedCommunities
    SET u:FraudRiskUser
    SET u.fraudRisk=1
    RETURN count(u)
''')

,count(u)
0,452


In [26]:
# This Cypher query assigns a fraudRisk score of 0 to all users who have not been 
# labeled as FraudRiskUser. This ensures that every User node in the database has a 
# defined fraud risk score, making it easier for downstream analysis.

"""
gds.run_cypher('''
    -- Step 1: Identify Users Who Are Not Marked as FraudRiskUser
    MATCH (u:User) 
    WHERE NOT u:FraudRiskUser  -- ✅ Select users who are NOT already labeled as high-risk fraud users

    -- Step 2: Assign a Fraud Risk Score of 0
    SET u.fraudRisk = 0  -- ✅ Explicitly mark these users as low-risk (non-fraudulent)

    -- Step 3: Return the Total Number of Users Updated
    RETURN count(u)  -- ✅ Output how many users were marked with `fraudRisk=0`
''')
"""

gds.run_cypher('''
    MATCH (u:User) WHERE NOT u:FraudRiskUser
    SET u.fraudRisk=0
    RETURN count(u)
''')

,count(u)
0,33280


In [29]:
# This Cypher query analyzes the distribution of fraud across different Weakly 
# Connected Components (WCC) of users. It groups communities (wccId) based on their 
# size, categorizes them into predefined bins (e.g., small, medium, large communities), 
# and calculates how many fraud-risk users exist in each size category.

"""
gds.run_cypher( '''
    -- Step 1: Group Users by Their Community (WCC ID) and Count Users in Each Community
    MATCH (u:User)
    WITH u.wccId AS community, count(u) AS cSize, sum(u.fraudRisk) AS cFraudSize
    -- ✅ `cSize` = Total number of users in the community
    -- ✅ `cFraudSize` = Number of users in the community who are flagged as fraud risk

    -- Step 2: Categorize Communities Based on Their Size
    WITH community, cSize, cFraudSize,
    CASE
        WHEN cSize = 1 THEN ' 1'               -- ✅ Isolated users (not connected to others)
        WHEN cSize = 2 THEN ' 2'               -- ✅ Small communities of two users
        WHEN cSize = 3 THEN ' 3'               -- ✅ Three-user communities
        WHEN cSize > 3 AND cSize <= 10 THEN ' 4-10'  -- ✅ Small clusters
        WHEN cSize > 10 AND cSize <= 50 THEN '11-50' -- ✅ Medium-sized clusters
        WHEN cSize > 50 THEN '>50'             -- ✅ Large communities (potential fraud rings)
    END AS componentSize  -- ✅ Assigns a size category label to each community

    -- Step 3: Count the Number of Communities in Each Size Category and Aggregate Fraud Data
    RETURN componentSize, 
        count(*) AS numberOfComponents,       -- ✅ Number of WCC communities in each size category
        sum(cSize) AS totalUserCount,         -- ✅ Total number of users in these communities
        sum(cFraudSize) AS fraudUserCount     -- ✅ Total number of flagged fraud users in these communities
    ORDER BY componentSize  -- ✅ Sort the results by community size category
''')

"""

gds.run_cypher( '''
    MATCH (u:User)
    WITH u.wccId AS community, count(u) AS cSize, sum(u.fraudRisk) AS cFraudSize
    WITH community, cSize, cFraudSize,
    CASE
        WHEN cSize=1 THEN ' 1'
        WHEN cSize=2 THEN ' 2'
        WHEN cSize=3 THEN ' 3'
        WHEN cSize>3 AND cSize<=10 THEN ' 4-10'
        WHEN cSize>10 AND cSize<=50 THEN '11-50'
        WHEN cSize>10 THEN '>50' END AS componentSize
    RETURN componentSize, 
        count(*) AS numberOfComponents, 
        sum(cSize) AS totalUserCount, 
        sum(cFraudSize) AS fraudUserCount 
    ORDER BY componentSize
''')

,componentSize,numberOfComponents,totalUserCount,fraudUserCount
0,1,24283,24283,118
1,2,3165,6330,144
2,3,481,1443,63
3,4-10,258,1246,68
4,11-50,15,255,59
5,>50,1,175,0


In [30]:
# This query calculates the proportion of P2P transaction volume that involves FraudRiskUsers. It does this by:
# 1. Summing the total amount of all P2P transactions involving at least one FlaggedUser.
# 2. Summing only the P2P transaction amount where a FraudRiskUser transacts with a FlaggedUser.
# 3. Computing the ratio of fraud-risk transactions to the total, representing how much of the P2P transaction volume is linked to high-risk users.

"""
gds.run_cypher('''
   -- Step 1: Compute Total P2P Transaction Amount Involving at Least One Flagged User
   MATCH (:FlaggedUser)-[r:P2P]-(u)  
   WHERE NOT u:FlaggedUser  -- ✅ Ensure the other party in the transaction is not flagged
   WITH toFloat(sum(r.totalAmount)) AS p2pTotal  -- ✅ Compute the total P2P transaction amount

   -- Step 2: Compute Fraud-Risk P2P Transactions (FraudRiskUser ↔ FlaggedUser)
   MATCH (u:FraudRiskUser)-[r:P2P]-(:FlaggedUser) 
   WHERE NOT u:FlaggedUser  -- ✅ Ensure the FraudRiskUser is not already flagged
   WITH p2pTotal, toFloat(sum(r.totalAmount)) AS fraudRiskP2pTotal  
   -- ✅ Compute total transaction volume where a FraudRiskUser is involved

   -- Step 3: Calculate the Proportion of FraudRisk P2P Transactions
   RETURN round((fraudRiskP2pTotal)/p2pTotal,3) AS p  -- ✅ Ratio of fraud-risk transactions to total P2P volume
''').p[0]  -- ✅ Extracts and returns the computed proportion value
"""

gds.run_cypher('''
   MATCH (:FlaggedUser)-[r:P2P]-(u)  WHERE NOT u:FlaggedUser
   WITH toFloat(sum(r.totalAmount)) AS p2pTotal
   MATCH (u:FraudRiskUser)-[r:P2P]-(:FlaggedUser) WHERE NOT u:FlaggedUser
   WITH p2pTotal,  toFloat(sum(r.totalAmount)) AS fraudRiskP2pTotal
   RETURN round((fraudRiskP2pTotal)/p2pTotal,3) AS p
''').p[0]

0.652

In [ ]:
# This query calculates the proportion of total P2P transaction volume that involves 
# FraudRiskUsers. It:

# 1. Computes the total amount of all P2P transactions in the dataset.
# 2. Computes the total amount of P2P transactions where a FraudRiskUser is involved 
# (excluding FlaggedUser).
# 3. Calculates the ratio of fraud-risk transactions to total P2P transactions.

"""gds.run_cypher('''
   -- Step 1: Compute Total P2P Transaction Amount
   MATCH (:User)-[r:P2P]->()
   WITH toFloat(sum(r.totalAmount)) AS p2pTotal  -- ✅ Compute total P2P transaction volume

   -- Step 2: Compute P2P Transactions Involving FraudRiskUsers (but NOT FlaggedUsers)
   MATCH (u:FraudRiskUser)-[r:P2P]-()  
   WHERE NOT u:FlaggedUser  -- ✅ Exclude transactions where the fraud risk user is already flagged
   WITH p2pTotal, toFloat(sum(r.totalAmount)) AS fraudRiskP2pTotal  
   -- ✅ Compute total transaction volume where a FraudRiskUser is involved

   -- Step 3: Calculate the Proportion of FraudRisk P2P Transactions
   RETURN round((fraudRiskP2pTotal)/p2pTotal,3) AS p  -- ✅ Ratio of fraud-risk transactions to total P2P volume
''').p[0]  -- ✅ Extracts and returns the computed proportion value
"""

gds.run_cypher('''
   MATCH (:User)-[r:P2P]->()
   WITH toFloat(sum(r.totalAmount)) AS p2pTotal
   MATCH (u:FraudRiskUser)-[r:P2P]-() WHERE NOT u:FlaggedUser
   WITH p2pTotal, toFloat(sum(r.totalAmount)) AS fraudRiskP2pTotal
   RETURN round((fraudRiskP2pTotal)/p2pTotal,3) AS p
''').p[0]

0.127

In [ ]:
# This code calculates the fraud risk ratio for financial and device identifiers (Card, Device, IP) by:

# 1. Computing the degree centrality (fraudRiskDegree) for FraudRiskUsers to measure their connections.
# 2. Setting a fraudRiskRatio for financial and device entities, which represents the proportion of their connections that are linked to fraud-risk users.

"""
# Step 1: Compute Degree Centrality for FraudRiskUsers
identifier_degrees('FraudRiskUser', 'fraudRiskDegree')
# ✅ Measures how many connections FraudRiskUsers have to other entities (Cards, Devices, IPs)
# ✅ Stores the computed value in a new property: `fraudRiskDegree`

# Step 2: Calculate Fraud Risk Ratio for Cards, Devices, and IPs
gds.run_cypher('''
    MATCH (n) 
    WHERE n:Card OR n:Device OR n:IP  -- ✅ Apply to financial and device identifiers

    -- Step 3: Compute fraudRiskRatio (fraudulent connections / total connections)
    SET n.fraudRiskRatio = toFloat(n.fraudRiskDegree)/toFloat(n.degree)
    -- ✅ `fraudRiskDegree` = How many connections this entity has to FraudRiskUsers
    -- ✅ `degree` = Total number of connections this entity has
    -- ✅ `fraudRiskRatio` = Percentage of fraudulent connections
''')

"""

identifier_degrees('FraudRiskUser', 'fraudRiskDegree')

gds.run_cypher('''
    MATCH(n) WHERE n:Card OR n:Device OR n:IP
    SET n.fraudRiskRatio = toFloat(n.fraudRiskDegree)/toFloat(n.degree)
''')

""


In [ ]:
# This query analyzes credit card fraud risk distribution by:

# 1. Filtering out cards used by only one user (degree > 1) to focus on shared cards.
# 2. Categorizing each card based on its fraudRiskRatio into three groups:
# "0" → No fraud-risk connections.
# "1" → All connections are to fraud-risk users.
# "Between 0-1" → Mixed fraud-risk connections.
# 3. Computing the proportion (percentCount) of cards in each category.

"""
gds.run_cypher('''
    -- Step 1: Compute the Total Number of Shared Cards
    MATCH (n:Card) 
    WHERE n.degree > 1  -- ✅ Consider only cards shared by multiple users
    WITH toFloat(count(n)) AS total  -- ✅ Store total count of shared cards

    -- Step 2: Categorize Cards Based on FraudRiskRatio
    MATCH (n:Card) 
    WHERE n.degree > 1  -- ✅ Re-select shared cards
    WITH n, total, 
         CASE 
             WHEN n.fraudRiskRatio = 0 THEN '0'  -- ✅ No fraud-risk users connected
             WHEN n.fraudRiskRatio = 1 THEN '1'  -- ✅ 100% of connections are fraud-risk users
             ELSE 'Between 0-1'  -- ✅ Mixed fraud-risk connections
         END AS fraudRiskRatio

    -- Step 3: Count Cards in Each Category
    WITH fraudRiskRatio, count(n) AS count, total  

    -- Step 4: Compute the Proportion of Cards in Each Category
    RETURN fraudRiskRatio, count, round(toFloat(count)/total,3) AS percentCount  
    ORDER BY fraudRiskRatio  -- ✅ Sort results by fraud risk category
''')

"""

gds.run_cypher('''
    MATCH (n:Card) 
    WHERE n.degree > 1
    WITH toFloat(count(n)) AS total
    
    MATCH (n:Card) 
    WHERE n.degree > 1
    WITH n, total, 
         CASE 
             WHEN n.fraudRiskRatio = 0 THEN '0'
             WHEN n.fraudRiskRatio = 1 THEN '1'
             ELSE 'Between 0-1' 
         END AS fraudRiskRatio
    WITH fraudRiskRatio, count(n) AS count, total
    RETURN fraudRiskRatio, count, round(toFloat(count)/total,3) AS percentCount 
    ORDER BY fraudRiskRatio
''')


,fraudRiskRatio,count,percentCount
0,0,8114,0.956
1,1,351,0.041
2,Between 0-1,25,0.003


PART 3.

In [33]:
gds.run_cypher('''
    MATCH(f:FraudRiskUser)-[:HAS_CC|HAS_IP|USED]->(n)
    WITH DISTINCT n
    MATCH(u:User)-[:HAS_CC|HAS_IP|USED]->(n) WHERE NOT u:FraudRiskUser
    WITH left(u.guid,8) as uid,
        sum(n.fraudRiskRatio) AS totalIdFraudRisk,
        count(n) AS numberFraudRiskIds
    WITH uid, totalIdFraudRisk,
        numberFraudRiskIds,
        totalIdFraudRisk/toFloat(numberFraudRiskIds) AS averageFraudIdRisk
    WHERE averageFraudIdRisk >= 0.25
    RETURN uid, totalIdFraudRisk, numberFraudRiskIds, averageFraudIdRisk
    ORDER BY totalIdFraudRisk DESC LIMIT 10
''')

,uid,totalIdFraudRisk,numberFraudRiskIds,averageFraudIdRisk
0,50fd11c0,7.405512,28,0.264483
1,b41b9e3b,6.487978,22,0.294908
2,a982e8fc,3.928543,13,0.302196
3,60b71ab9,2.020481,7,0.288640
4,bc1eb835,2.010309,5,0.402062
5,1b1a78a9,1.925455,5,0.385091
6,f2b8a068,1.693794,6,0.282299
7,1c773976,1.594978,5,0.318996
8,a2f748e3,1.570455,4,0.392614
9,7524a090,1.527338,5,0.305468


In [34]:
# label identifiers and users that are close to fraud risk users and assign inverse degree weight
gds.run_cypher('''
    MATCH(f:FraudRiskUser)-[:HAS_CC|HAS_IP|USED]->(n)
    WITH DISTINCT n
    MATCH(n)<-[r:HAS_CC|HAS_IP|USED]-(u)
    SET n:FraudSharedId
    SET r.inverseDegreeWeight = 1.0/(n.degree-1.0)
    RETURN count(DISTINCT n)
''')

,count(DISTINCT n)
0,18182


In [35]:
# clear the graph if it exists beforehand
clear_graph_by_name('similarity-projection')

g, _ = gds.graph.project('similarity-projection', ['User', 'FraudSharedId'], ['HAS_CC', 'USED', 'HAS_IP'],
                         relationshipProperties=['inverseDegreeWeight'])



df = gds.nodeSimilarity.write(g, writeRelationshipType='SIMILAR_IDS', writeProperty='score',
                              similarityCutoff=0.01, relationshipWeightProperty='inverseDegreeWeight')
g.drop()
df

preProcessingMillis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        15
computeMillis                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [36]:
#get nodes similar to the high risk ones
gds.run_cypher('''
    MATCH (f:FraudRiskUser)
    WITH f.wccId AS componentId, count(*) AS numberOfUsers, collect(f) AS users
    UNWIND users AS f
    MATCH (f)-[s:SIMILAR_IDS]->(u:User) WHERE NOT u:FraudRiskUser AND numberOfUsers > 2
    RETURN u.guid AS userId, sum(s.score) AS totalScore, collect(DISTINCT componentId) AS closeToCommunityIds 
    ORDER BY totalScore DESC
''')

,userId,totalScore,closeToCommunityIds
0,3b4284c78a2c54de93012d0e0da83414,0.780392,[433]
1,0b3f278ff6b348fb1a599479d9321cd9,0.527056,"[4616, 4932]"
2,99d857cf888723db4ab2c19e42ac2b4c,0.367563,[4616]
3,c0d80bb57c0bbb0b68f647c7adc83274,0.298319,[4932]
4,d1c8aaa603264c33fbc370c0262cdd83,0.246118,[433]
5,4b93a079eb3ff279af0c53d8797e3899,0.215148,[2130]
6,6b41e66c7667df06c10eab6c6b712fec,0.201825,[4932]
7,3655349eda094010fe12e7d8650187e4,0.192911,[433]
8,168e0a7f0642adbeef8d9c5753dd6dfa,0.162240,[433]
9,55447b3000b48335d3b18557c43ba70a,0.136346,[4616]
